Description of data set found here: https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

What is the likelihood of default between categories of different demographics variables?


In [23]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
data = pd.read_excel("default of credit card clients.xls", skiprows = 1)

In [27]:
data = data.set_index('ID')
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [28]:
#relabeling data for clarity
data.EDUCATION = data.EDUCATION.map({1: 'graduate school', 2: 'university', 3: 'high school', 4: 'others'})
data.MARRIAGE = data.MARRIAGE.map({1: 'married', 2: 'single', 3: 'others'})
data.SEX = data.SEX.apply(lambda x: "MALE" if x == 1 else "FEMALE")

In [29]:
data.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,FEMALE,university,married,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,FEMALE,university,single,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,FEMALE,university,single,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,FEMALE,university,married,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,MALE,university,married,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
print "number of rows with null values: ", len(data[data.isnull().any(axis=1)])
data = data.dropna(axis = 0, how = 'any')

number of rows with null values:  399


In [7]:
print "number of \"others\" under MARRIAGE or EDUCATION:",len(data[data.MARRIAGE == 'others']) + len(data[data.EDUCATION == 'others'])
data = data[data.MARRIAGE != 'others'][data.EDUCATION != 'others']

number of "others" under MARRIAGE or EDUCATION: 441


C:\Users\Alvin\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [8]:
#function to print out median debt and percentage that default based on a group
def demostats(df, group):
    gp = df.groupby(group)
    print "Median Credit Limit\n"
    print gp.LIMIT_BAL.median()
    print "\nTotal number of default payments/Total number of loans"
    print "\n", gp['default payment next month'].mean()

In [9]:
#Group by sex
demostats(data, 'SEX')

Median Credit Limit

SEX
FEMALE    150000
MALE      130000
Name: LIMIT_BAL, dtype: int64

Total number of default payments/Total number of loans

SEX
FEMALE    0.210200
MALE      0.243369
Name: default payment next month, dtype: float64


In [10]:
#Group by education
demostats(data, 'EDUCATION')

Median Credit Limit

EDUCATION
graduate school    200000
high school         80000
university         110000
Name: LIMIT_BAL, dtype: int64

Total number of default payments/Total number of loans

EDUCATION
graduate school    0.192100
high school        0.252830
university         0.236979
Name: default payment next month, dtype: float64


In [11]:
#group by age bracket
def agebracket(x):
    if x >= 18 and x <= 24:
        return "18-24"
    elif x >= 25 and x <= 34:
        return "25-34"
    elif x >= 35 and x <= 44:
        return "35-44"
    elif x >= 45 and x <= 54:
        return "45-54"
    elif x >= 55:
        return "55+"
    
data['Age Bracket'] = data.AGE.apply(lambda x: agebracket(x))
demostats(data, 'Age Bracket')

Median Credit Limit

Age Bracket
18-24     50000
25-34    150000
35-44    180000
45-54    140000
55+      130000
Name: LIMIT_BAL, dtype: int64

Total number of default payments/Total number of loans

Age Bracket
18-24    0.275076
25-34    0.205191
35-44    0.220560
45-54    0.241972
55+      0.268657
Name: default payment next month, dtype: float64


In [12]:
#By all combinations of sex, education, and age bracket
demostats(data, ['SEX', 'MARRIAGE', 'EDUCATION', 'Age Bracket'])
#gp = data.groupby(['EDUCATION'])
#gp['default payment next month'].size()

Median Credit Limit

SEX     MARRIAGE  EDUCATION        Age Bracket
FEMALE  married   graduate school  18-24           50000
                                   25-34          210000
                                   35-44          230000
                                   45-54          235000
                                   55+            250000
                  high school      18-24           45000
                                   25-34           90000
                                   35-44          110000
                                   45-54           80000
                                   55+             80000
                  university       18-24           50000
                                   25-34          130000
                                   35-44          150000
                                   45-54          140000
                                   55+            140000
        single    graduate school  18-24           50000
                    

In [13]:
from operator import itemgetter
gp = data.groupby(['SEX','MARRIAGE','EDUCATION','Age Bracket'])

#create lists of tuples
credit_list = [] 
default_rate_list = []

for i,j in gp:
    credit = round(j.LIMIT_BAL.median(), 0)
    credit_list.append((i, credit))
    default_rate = round(j['default payment next month'].mean(),3)
    default_rate_list.append((i, default_rate))

credit_list = sorted(credit_list, key = lambda x: x[1])    
default_rate_list = sorted(default_rate_list, key = lambda x: x[1])

In [14]:
print "Highest Credit Limit:", credit_list[-1]
print "Lowest Credit Limit:", credit_list[0]
print "Highest Rate of Default:", default_rate_list[-1]
print "Lowest Rate of Default:", default_rate_list[0]

Highest Credit Limit: (('MALE', 'married', 'graduate school', '55+'), 285000.0)
Lowest Credit Limit: (('MALE', 'single', 'high school', '18-24'), 20000.0)
Highest Rate of Default: (('MALE', 'married', 'high school', '18-24'), 0.444)
Lowest Rate of Default: (('MALE', 'single', 'university', '55+'), 0.158)


In [15]:
#Next steps, derive fields like amount owed and amount paid, amount owed/credit limit